In [ ]:
#impor packages
import pandas as pd
import numpy as np
import os
import pickle
import json
import re

In [ ]:
# location of raw data files
input_path = "/home/jupyter/datasets/raw/"

In [ ]:
#read PRESCRIPTIONS.csv file
df = pd.read_csv(filepath_or_buffer=input_path+'PRESCRIPTIONS.csv',dtype = "object")

In [ ]:
df.info(verbose=True, null_counts=True)

In [ ]:
df.head()

In [ ]:
df['GSN'] = 'GSN'+'-'+df['GSN']
df['NDC'] = 'NDC'+'-'+df['GSN']


In [ ]:
df['FORMULARY_DRUG_CD'] = 'DRUG_CD'+'-'+df['FORMULARY_DRUG_CD']

In [ ]:
df.head(1000)

In [ ]:
# Drop DRUG_NAME_POE as both are the same data
df = df.drop(labels = ['DRUG_NAME_POE'], axis = 'columns')

In [ ]:
df = df.fillna('NaN')

In [ ]:
# replace white space in product stength with '-'
df['PROD_STRENGTH'] = df['PROD_STRENGTH'].map(lambda x: '-'.join(str(x).split()))


In [ ]:
# Merge Dosage VALUE and Dosage UNIT 
df['DOSE_UNIT_VAL_RX'] = df['DOSE_VAL_RX']+'-'+df['DOSE_UNIT_RX']
df.loc[df['DOSE_VAL_RX']=='NaN','DOSE_UNIT_VAL_RX'] = 'NaN'

In [ ]:
# Merge FORM VALUE and FORM UNIT
df['FORM_VAL_UNIT_DISP'] = df['FORM_VAL_DISP']+'-'+df['FORM_UNIT_DISP']
df.loc[df['FORM_VAL_DISP']=='NaN','FORM_VAL_UNIT_DISP'] ='NaN'

In [ ]:
#Drop redundant columns
df = df.drop(labels = ['DOSE_VAL_RX','DOSE_UNIT_RX','FORM_VAL_DISP','FORM_UNIT_DISP'], axis = 'columns')

In [ ]:
# Replace special charecters in free text
df['DRUG'] = df['DRUG'].map(lambda x : re.sub('[!"#$&()*+,;<=>?@[\]^_`{|}~\n]', '', x))
df['DRUG_NAME_GENERIC'] = df['DRUG_NAME_GENERIC'].map(lambda x : re.sub('[!"#$&()*+,;<=>?@[\]^_`{|}~\n]', '', x))

In [ ]:
# replace white space in drug with '-' 
df['DRUG_CONCAT'] = df['DRUG'].map(lambda x: '-'.join(str(x).split()))

In [ ]:
df.head()

In [ ]:
# Concat PRODUCt STREGNTH, DOSGAE, FORM with DRUGS
df['DRUG_PROD_STRENGTH'] = df['DRUG_CONCAT']+'-'+df['PROD_STRENGTH']
df['DRUG_DOSE_UNIT_VAL_RX'] = df['DRUG_CONCAT']+'-'+df['DOSE_UNIT_VAL_RX']
df['DRUG_FORM_VAL_UNIT_DISP'] = df['DRUG_CONCAT']+'-'+df['FORM_VAL_UNIT_DISP']

In [ ]:
df = df.drop(labels=['DRUG_CONCAT','PROD_STRENGTH','DOSE_UNIT_VAL_RX','FORM_VAL_UNIT_DISP'], axis = 'columns')

In [ ]:
df.head()

In [ ]:
df['prescriptionevents'] = df['DRUG_TYPE']+' '+df['DRUG']+' '+df['DRUG_NAME_GENERIC']+' '+df['FORMULARY_DRUG_CD']+' '+df['GSN']+' '+df['NDC']+' '+df['ROUTE']+' '+df['DRUG_PROD_STRENGTH']+' '+df['DRUG_DOSE_UNIT_VAL_RX']+' '+df['DRUG_FORM_VAL_UNIT_DISP']

In [ ]:
df = df.drop(labels=['DRUG_TYPE','DRUG','DRUG_NAME_GENERIC','FORMULARY_DRUG_CD',
                    'GSN','NDC','ROUTE','DRUG_PROD_STRENGTH','DRUG_DOSE_UNIT_VAL_RX',
                     'DRUG_FORM_VAL_UNIT_DISP'], axis = 'columns')

In [ ]:
df.head()

In [ ]:
df['prescriptionevents'] = df['prescriptionevents'].map(lambda x: x.split())

In [ ]:
# remove 'NaN' form all events
df['prescriptionevents'] = df['prescriptionevents'].map(lambda x: list(filter(lambda a: a!='NaN',x)))

In [ ]:
df.head()

In [ ]:
# create output path
mypath_output = "../datasets/prescriptionevents/tokenized/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [ ]:
# export the dataframe to JSON format
df.to_json(mypath_output+"prescriptionevents.json",orient = 'records') 